In [1]:
import pandas
from IPython.display import clear_output

In [2]:
from classes import Question
import look_up_utils
import createq_utils
from display_utils import display_question

## Set parameter values

In [3]:
from glob import glob
import os
frame_paths = [os.path.basename(path)
              for path in glob('../EventRegistries/GunViolence/frames/mass*')
              if not 'Icon' in path
             ]

add_fr = False
if add_fr:
    frame_paths.append('../EventRegistries/FireRescue/firerescue.pickle')

print(frame_paths)

df = pandas.concat([pandas.read_pickle('../EventRegistries/GunViolence/frames/' + frame)
                    for frame in frame_paths])

confusion_tuple = ('location', 'participant')
min_num_answer_incidents = 2

['mass_shootings', 'mass_shootings_2013', 'mass_shootings_2014', 'mass_shootings_2015']


## Load frames

## Load look_up

In [4]:
look_up, parameters2incident_uris = look_up_utils.create_look_up(df, discard_ambiguous_names=True)

## Load questions

In [5]:
candidates=createq_utils.lookup_and_merge(look_up, 
                                          parameters2incident_uris,
                                          confusion_tuple,
                                          min_num_answer_incidents,
                                          df,
                                          debug=False,
                                          inspect_one=False) 
print({c.granularity for c in candidates})

{('city', 'full_name'), ('state', 'first'), ('city', 'last'), ('state', 'full_name'), ('city', 'first'), ('state', 'last')}


## Display a question

In [6]:
print('the available attributes are:\n')
for candidate in candidates:
    for prop in dir(candidate):
        if all([not prop.startswith('_'),
                prop not in {'debug', 'confusion_df', 'answer_df'}]):
            print(prop)
    break

the available attributes are:

a_avg_num_sources
a_sources
answer
answer_incident_uris
c2s_ratio
c_avg_num_sources
c_sources
confusion_factors
confusion_incident_uris
gold_loc_meaning
granularity
loc_oa
location_confusion
meanings
num_both_sf_overlap
oa
oa_info
participant_confusion
q_id
question
sf
time_confusion
time_oa


In [9]:
wanted_attrs = [
    'q_id',
    'question',
    'granularity',
    'answer_incident_uris',
    'num_both_sf_overlap',
    #'confusion_incident_uris',
    'meanings',
    'answer',
    'c2s_ratio',
    'location_confusion',
    'time_confusion',
    'participant_confusion',
    'a_avg_num_sources',
    #'a_avg_date_spread',
    'c_avg_num_sources',
    #'c_avg_date_spread',
    'loc_oa',
    'time_oa'
]

for candidate in candidates:
    clear_output()
    display_question(candidate, wanted_attrs)
    input('next?')

,0
q_id,115
question,"How many killing AND injuring events happened in ('Tennessee',) (state) that involve the name Malone (last) ?"
granularity,"('state', 'last')"
answer_incident_uris,"{'162008', '691113'}"
num_both_sf_overlap,2
meanings,"defaultdict(, {(('Tennessee',), ('691113', 'Malone')): {'691113'}, (('Tennessee',), ('162008', 'Malone')): {'162008'}})"
answer,2
c2s_ratio,24.0
location_confusion,True
time_confusion,False


KeyboardInterrupt: 